In [1]:
import numpy as np
import random
from platypus import Hypervolume, display, calculate, CMAES, MOEAD, NSGAII, NSGAIII, SPEA2, IBEA, Problem, Real
from copy import deepcopy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd

from datetime import datetime

In [2]:
from load_data import *
from bias_functions import *
from utility_functions import *

In [3]:
def getDataset(setSelection):
    if setSelection == 'toy':
        X,y = generate_toy_data(1000,200,2)
    elif setSelection == 'adult':
        protectedAttributes={'race':'White','gender':'Male'}
        X,y = load_adult(protectedAttributes=protectedAttributes)
    elif setSelection == 'bank':
        X,y = load_bank()
    elif setSelection == 'german':
        X,y = load_german()
    elif setSelection == 'mortgage':
        protectedCategoricalAttributes={'applicant_ethnicity_name':'Not Hispanic or Latino',
                                'applicant_race_name_1':'White','applicant_sex_name':'Male'}
        protectedNumericalAttributes=['minority_population']
        X,y = load_mortgage(protectedCategoricalAttributes=protectedCategoricalAttributes, \
                            protectedNumericalAttributes=protectedNumericalAttributes)
    else:
        print('dataset not recognised')
        
    X = np.hstack([X, np.ones((X.shape[0],1))]) ## add ones to solve for affine functions
    
    return X,y

In [4]:
# new problems

# problem 5: datasets {Adult, German, Mortgage} and {age,gender,race} [change dataset in main loop ; change senstitive attributes in _train and _test functions]
# objectives: accuracy, DI, EO, DM(OMR) across 2 attributes

def problem8_base(w,X,y,sensitiveAttributeIndex1,sensitiveAttributeIndex2):
    errorRateObjective = errorRate(w,X,y)
    fairnessObjective1 = differenceDisparateImpactModel(w,X,sensitiveAttributeIndex=sensitiveAttributeIndex1)
    fairnessObjective2 = differenceDisparateImpactModel(w,X,sensitiveAttributeIndex=sensitiveAttributeIndex2)
    fairnessObjective3 = differenceEqualOpportunity(w,X,y,sensitiveAttributeIndex=sensitiveAttributeIndex1)
    fairnessObjective4 = differenceEqualOpportunity(w,X,y,sensitiveAttributeIndex=sensitiveAttributeIndex2)
    fairnessObjective5 = differenceDisparateMistreatment(w,X,y,sensitiveAttributeIndex=sensitiveAttributeIndex1,type='OMR')
    fairnessObjective6 = differenceDisparateMistreatment(w,X,y,sensitiveAttributeIndex=sensitiveAttributeIndex2,type='OMR')
    return errorRateObjective,\
max(np.abs(fairnessObjective1[0]-fairnessObjective1[1]),np.abs(fairnessObjective2[0]-fairnessObjective2[1])),\
max(np.abs(fairnessObjective3[0]-fairnessObjective3[1]),np.abs(fairnessObjective4[0]-fairnessObjective4[1])),\
max(np.abs(fairnessObjective5[0]-fairnessObjective5[1]),np.abs(fairnessObjective6[0]-fairnessObjective6[1]))

def problem8_train(w):
    return problem8_base(w,trainxs,trainys,sensitiveAttributeIndex1=sensitiveAttributeIndex1,sensitiveAttributeIndex2=sensitiveAttributeIndex2)
def problem8_test(w):
    return problem8_base(w,testxs,testys,sensitiveAttributeIndex1=sensitiveAttributeIndex1,sensitiveAttributeIndex2=sensitiveAttributeIndex2)

In [5]:
targetValue = 0.01
targetMeasureArray = np.array([[np.inf,np.inf,np.inf],[targetValue,np.inf,np.inf],\
                             [np.inf,targetValue,np.inf],[np.inf,np.inf,targetValue],\
                             [targetValue,targetValue,np.inf],[targetValue,np.inf,targetValue],\
                             [np.inf,targetValue,targetValue],[targetValue,targetValue,targetValue]])

## Sensitive attributes list:

adult: 8 - race, 9 - gender ;  variables = 15 ; 45222 samples

bank: 0 - age ; variables = 21 ; 41188 samples

german: 6 - gender, 9 - age, 14 - foreign worker ; variables = 25 ; 1000 samples

mortgage: 12 - ethnicity, 14 - race, 16 - gender, 24 - minority population ; variables = 30 ; 200000 samples

In [6]:
now = datetime.now()

# these lines define the problem, test problem, dataset, algorithm and number of sampling runs
noOfObjectives = 4

dataset = 'mortgage'

if dataset == 'adult':
    sensitiveAttributeIndex1 = 9
    sensitiveAttributeIndex2 = 8
    noOfVariables = 15
elif dataset == 'german':
    sensitiveAttributeIndex1 = 9
    sensitiveAttributeIndex2 = 6
    noOfVariables = 25
else:
    sensitiveAttributeIndex1 = 16
    sensitiveAttributeIndex2 = 14
    noOfVariables = 30
    
problem = Problem(noOfVariables,noOfObjectives)
problem.function = problem8_train

testProblem = Problem(noOfVariables,noOfObjectives)
testProblem.function = problem8_test



noOfSamplingRuns = 20

X,y = getDataset(dataset)

problem.types[:] = Real(-5,5)

hypervolumeArray = np.zeros((noOfSamplingRuns, 2)) # col 0 for training results (hypervolume), col 1 for test results
extremesArray = np.zeros((2, noOfObjectives, noOfObjectives, noOfSamplingRuns)) # the first dimension is for train/test
averagePointArray = np.zeros((2, noOfSamplingRuns, noOfObjectives)) # the first dimension is for train/test. This array holds the average objective point for each run

overallErrorRateSet = np.zeros((2, noOfSamplingRuns, len(targetMeasureArray)))

# these will be used to store the algorithm that returns the largest hypervolume
bestTrainAlgorithm = None
bestTrainHypervolume = 0
bestTestAlgorithm = None
bestTestHypervolume = 0
totalTrainTime = 0
totalTestTime = 0

for run in range(noOfSamplingRuns):

    # generate new train/test split for each run
    trainxs, testxs, trainys, testys  = train_test_split(X,y,train_size=0.8)  
    
    # training run
    startTime = time.perf_counter()
    
    trainAlgorithm = CMAES(problem)
    trainAlgorithm.run(10000)

    trainHyp = Hypervolume(minimum=[0]*noOfObjectives,maximum=[1]*noOfObjectives)
    trainHypResult = trainHyp(trainAlgorithm.result)  
    hypervolumeArray[run, 0] = trainHypResult
    
    singleRunExtremesArray, averagePoint = findExtremes_averagePoint(trainAlgorithm)
    extremesArray[0,:,:,run] = singleRunExtremesArray
    averagePointArray[0,run,:] = averagePoint
    
    minErrorRateSet = minErrorRateSet_given_targetMeasuresArray(trainAlgorithm, targetMeasureArray)
    overallErrorRateSet[0,run,:] = minErrorRateSet[:,0]
    
    endTime = time.perf_counter()
    trainTime = endTime - startTime
    totalTrainTime += trainTime
    
    print("{}: training took {:.2f} seconds w/ hypervolume {:2f}".format(run, trainTime, trainHypResult))
    
    
    # test run
    testAlgorithm = deepcopy(trainAlgorithm)

    startTime = time.perf_counter()

    for resultNumber, result in enumerate(testAlgorithm.result):

        w=result.variables
        objectivesResult = testProblem.function(w)

        # copy objective results individually to the testAlgorithm object
        for i in range(len(objectivesResult)):  
            testAlgorithm.result[resultNumber].objectives[i]=objectivesResult[i]

    testHyp = Hypervolume(minimum=[0]*noOfObjectives,maximum=[1]*noOfObjectives)
    testHypResult = testHyp(testAlgorithm.result)
    hypervolumeArray[run, 1] = testHypResult
    
    singleRunExtremesArray, averagePoint = findExtremes_averagePoint(testAlgorithm)
    extremesArray[1,:,:,run] = singleRunExtremesArray
    averagePointArray[1,run,:] = averagePoint
    
    for targetMeasure in range(len(targetMeasureArray)):
        if minErrorRateSet[targetMeasure,0] == 1.0:
            overallErrorRateSet[1,run,targetMeasure] = 1.0
        else:
            overallErrorRateSet[1,run,targetMeasure] = testAlgorithm.result[np.int(minErrorRateSet[targetMeasure,1])].objectives[0]
    
    
    endTime = time.perf_counter()
    testTime = endTime - startTime
    totalTestTime += testTime

    print("{}: testing took {:.2f} seconds w/ hypervolume {:2f}".format(run, testTime, testHypResult))
    
    if trainHypResult > bestTrainHypervolume:
        bestTrainHypervolume = trainHypResult
        bestTrainAlgorithm = deepcopy(trainAlgorithm)
        
        bestTestHypervolume = testHypResult
        bestTestAlgorithm = deepcopy(testAlgorithm)
        
        bestData = [trainxs, testxs, trainys, testys]
        bestDataSavePath = "saved_data/"+now.strftime("%Y%m%d_%H%M")+"_"+str(problem.function)[10:18]+"_"+dataset+"_"+str(trainAlgorithm)[21:-26]+"_data"
        np.save(bestDataSavePath, bestData)
        
# resultsSummary:
# column 0: average hypervolume, column 1: hypervolume std, column 2: average point std (Euclidean distance), column 3: average time
# row 0: training data, row 1: test data
resultsSummary = np.zeros((2,4)) 
resultsMean = np.sum(hypervolumeArray, axis=0)/noOfSamplingRuns 
resultsSummary[:,0] = resultsMean
resultsSummary[:,1] = np.sqrt(np.sum((hypervolumeArray - resultsMean)**2, axis=0)/noOfSamplingRuns)
averagePointMeans = np.mean(averagePointArray,axis=1)
resultsSummary[0,2] = np.std(np.sqrt(np.sum((averagePointArray[0,:,:]-averagePointMeans[0,:])**2,axis=1)))
resultsSummary[1,2] = np.std(np.sqrt(np.sum((averagePointArray[1,:,:]-averagePointMeans[1,:])**2,axis=1)))
resultsSummary[0,3] = totalTrainTime/noOfSamplingRuns
resultsSummary[1,3] = totalTestTime/noOfSamplingRuns
print(resultsSummary) 

(200000, 30)
(200000, 29)
Protected Categorical Attributes:
applicant_ethnicity_name: 12
applicant_race_name_1: 14
applicant_sex_name: 16
Protected Numerical Attributes:
minority_population: 24
The dataset is loaded...
0: training took 614.25 seconds w/ hypervolume 0.854966
0: testing took 32.60 seconds w/ hypervolume 0.853205


/Users/nigelkingsman/opt/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


1: training took 593.92 seconds w/ hypervolume 0.862134
1: testing took 55.68 seconds w/ hypervolume 0.863481
2: training took 1905.46 seconds w/ hypervolume 0.861489
2: testing took 208.49 seconds w/ hypervolume 0.861940
3: training took 1214.36 seconds w/ hypervolume 0.865146
3: testing took 340.52 seconds w/ hypervolume 0.863374
4: training took 8572.56 seconds w/ hypervolume 0.857015
4: testing took 92.57 seconds w/ hypervolume 0.854128
5: training took 1029.33 seconds w/ hypervolume 0.868115
5: testing took 140.96 seconds w/ hypervolume 0.868456
6: training took 629.62 seconds w/ hypervolume 0.858276
6: testing took 43.80 seconds w/ hypervolume 0.854485
7: training took 659.48 seconds w/ hypervolume 0.861604
7: testing took 75.44 seconds w/ hypervolume 0.864474
8: training took 682.09 seconds w/ hypervolume 0.866463
8: testing took 82.06 seconds w/ hypervolume 0.866364
9: training took 734.13 seconds w/ hypervolume 0.867269
9: testing took 116.30 seconds w/ hypervolume 0.861341
10